In [6]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

client = Client(API_KEY, API_SECRET)

In [36]:
import sqlite3

db = sqlite3.connect('crypto.db')

cursor = db.cursor()

cursor.execute("""
    CREATE TABLE test (
        symbol TEXT,
        avg_price REAL,
        qty INTEGER,
        buy_time REAL
    )""")
cursor.execute("""
    CREATE TABLE block (
        symbol TEXT,
        unblocktime REAL
    )""")
db.commit()
db.close()

In [73]:
def fetchall(table='test'):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT * FROM {table}")
    alldata = cursor.fetchall()
    db.commit()
    db.close()
    return alldata

def extract_from_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def execute(command):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(command)
    db.commit()
    db.close()
    
def add_to_db(symbol, price, qty):
    time = int(datetime.utcnow().strftime('%d%H%M%S'))
    execute(f"INSERT INTO test VALUES ('{symbol}',{price},{qty},{time})")
    print(f"Bought {qty} units of {symbol} at ${price:.2f}")

def delete_from_db(symbol,price,qty,profit):
    # execute(f"DELETE FROM test WHERE symbol = '{symbol}'")
    print(f"Sold {qty} units of {symbol} at ${price:.2f}. Profit {profit:.2f}%")
    
def get_data(symbol,table='test'):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT * FROM {table} WHERE symbol = '{symbol}'")
    data = cursor.fetchone()
    db.commit()
    db.close()
    return data

def extract_from_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def block(symbol):
    time = int((datetime.utcnow() + timedelta(hours=1)).strftime('%d%H%M%S'))
    execute(f"INSERT INTO block VALUES ('{symbol}',{time})")

In [ ]:
# date_1 = datetime.utcnow().strftime('%y %m %d %H:%M:%S.%f')
# date_2 = datetime.utcnow().strftime('%d%H%M%S%f')
date_1 = datetime.utcnow().strftime('%y %m %d %H:%M:%S')
date_2 = datetime.utcnow().strftime('%d%H%M%S')
print(date_1)
print(int(date_2))